In [2]:
import numpy as np

# fonction de décomposition d'une matrice carrée 𝐴=𝐿+𝐷+𝑈 avec 𝐷 la matrice diagonale constituée des éléments diagonaux de 𝐴, 𝑈 et 𝐿 des matrices triangulaires respectivement supérieure et inférieure dont les éléments diagonaux sont nuls.

In [2]:
def decompose_LDU(A):
    #creation de masque de matrice
    mask_D = np.fromfunction(lambda i, j : i==j, A.shape) 
    mask_L = np.fromfunction(lambda i, j : i>j, A.shape)
    mask_U = np.fromfunction(lambda i, j : i<j, A.shape)

    # extraction des valeur souhaité grace au mask
    D = np.zeros(A.shape)
    D[mask_D] = A[mask_D]

    L = np.zeros(A.shape)
    L[mask_L] = A[mask_L]

    U = np.zeros(A.shape)
    U[mask_U] = A[mask_U]
    return L, D, U





A = np.random.randint(1, 10, (4, 4))
L, D, U = decompose_LDU(A)
print(L)
print(D)
print(U)

[[0. 0. 0. 0.]
 [4. 0. 0. 0.]
 [9. 3. 0. 0.]
 [7. 5. 7. 0.]]
[[2. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 2.]]
[[0. 5. 7. 4.]
 [0. 0. 5. 9.]
 [0. 0. 0. 7.]
 [0. 0. 0. 0.]]


In [3]:
def matrice_prod(A, B): 
    assert A.shape[0] == B.shape[1], "Taille des matrices non coherantes"
    
    return np.array([[np.sum(A[i]*B[:,j])  for j in range(B.shape[1])  ] for i in range(A.shape[0])])


A = np.random.randint(1, 10, (2, 2))
B = np.random.randint(1, 10, (2, 2))

print(matrice_prod(A, B))
print(A@B)

[[27 34]
 [20 30]]
[[27 34]
 [20 30]]


In [4]:
%timeit matrice_prod(A, B)

88.3 µs ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
%timeit np.dot(A, B)

4.49 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


62.6 µs ± 3.92 pour ma fonction contre 2.95 µs ± 977 ns pour numpy

On remarque qu'avec la methode des rectangles on par defaut une surestimation de l'integral.
Et plus le nombre de subdivision augmente plus 

## Matrice Elementaire

In [14]:
def T(i, j, n):
    assert i <= n and j <= n
    A = np.eye(n)
    i, j = i-1, j-1
    temp = A[i].copy()
    A[i] = A[j]
    A[j] = temp
    return A

def D(alpha, i,n):
    assert i <= n
    i = i-1
    A = np.eye(n)
    A[i, i] = alpha
    return A

def L(alpha, i, j, n):
    assert i < n and j < n
    A = np.eye(n)
    # i, j = i-1, j-1
    A [i, j] = alpha
    return (A)





## invDr(λ) = Dr(1/λ) 

In [7]:
alpha = 2
d = D(alpha, 3, 4) 
np.all(np.linalg.inv(d) == D(1/alpha, 3, 4))

True

## invLrs (λ) = Lrs (−λ)

In [8]:
alpha = 2
l = L(alpha, 3, 2, 4) 
np.all(np.linalg.inv(d) == L(-alpha, 3, 2, 4) )

False

## invTij = Tij

In [9]:
alpha = 2
t32= T( 3, 2, 4) 
np.all(np.linalg.inv(t32) == t32 )

True

# Verification des propreités du produit A = Lr1,s(λ1)*Lr2,s(λ2)

In [10]:
n= 4 # on va travailler sur une matrice 4x4
r1 = 2
r2 = 3
s = 1

lambda1, lambda2 = 3, 4
l1= L(lambda1, r1, s, n)
l2 = L(lambda2, r2, s, n)

A = l1@l2

print(A)
print()
print(A[r1-1, s-1] == lambda1)
print(A[r2-1, s-1] == lambda2)

[[1. 0. 0. 0.]
 [3. 1. 0. 0.]
 [4. 0. 1. 0.]
 [0. 0. 0. 1.]]

True
True


# Génération de la matrice 𝑀𝑟((𝜆𝑟+1,…,𝜆𝑛))=𝐿𝑟+1,𝑟(𝜆𝑟+1)…𝐿𝑛,𝑟(𝜆𝑛)

In [5]:
A = np.random.randint(1, 10, (4, 4))

print(A)

[[4 3 2 6]
 [6 7 2 6]
 [2 9 8 7]
 [2 6 8 8]]


In [6]:
def generate_Mr_down(M, r , n):
    assert 1 <= r <=n , print(f"r : {r}, n : {n}")
    lambda_list = M[r:, r-1]
    A = np.eye(n)
    Ap = np.eye(n)
    for row in range(r+1, n+1):
        A = A@L(-lambda_list[row-r-1]/M[r-1, r-1], row, r, n)
        Ap = Ap@L(lambda_list[row-r-1]/M[r-1, r-1], row, r, n)
    return A, Ap



r = 3 # ligne 1 index 0
Mdk = [generate_Mr_down(A, r,  A.shape[0])  for r in range(1, A.shape[0]+1)]
for m in Mdk:
    print(m)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [13]:
def generate_Mr_up(M, r , n):
    assert 1 <= r <=n , print(f"r : {r}, n : {n}")
    lambda_list = M[:r-1, r-1]
    A = np.eye(n)
    Ap = np.eye(n)
    for row in range(r-2, -1, -1):
        A = A@L(-lambda_list[row]/M[r-1, r-1], row+1, r, n)
        Ap = Ap@L(lambda_list[row]/M[r-1, r-1], row+1, r, n)
    return A, Ap

r = 3 # ligne 1 index 0
Muk = [generate_Mr_up(A, r,  A.shape[0])  for r in range(1, A.shape[0]+1)]
for muk in  Muk :
    print(muk)

(array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]))
(array([[ 1.  , -1.25,  0.  ,  0.  ],
       [ 0.  ,  1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ]]), array([[1.  , 1.25, 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ]]))
(array([[ 1. ,  0. , -1. ,  0. ],
       [ 0. ,  1. , -1.6,  0. ],
       [ 0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ]]), array([[1. , 0. , 1. , 0. ],
       [0. , 1. , 1.6, 0. ],
       [0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 1. ]]))
(array([[ 1.  ,  0.  ,  0.  , -0.5 ],
       [ 0.  ,  1.  ,  0.  , -0.75],
       [ 0.  ,  0.  ,  1.  , -0.75],
       [ 0.  ,  0.  ,  0.  ,  1.  ]]), array([[1.  , 0.  , 0.  , 0.5 ],
       [0.  , 1.  , 0.  , 0.75],
       [0.  , 0.  , 1.  , 0.75],
       